# VICREG

## Setup

In [1]:
! nvidia-smi

Wed Nov 27 12:53:43 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.256.02   Driver Version: 470.256.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN Xp     Off  | 00000000:01:00.0 Off |                  N/A |
| 28%   53C    P0    81W / 250W |      2MiB / 12196MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
import torch.distributed as dist

def off_diagonal(x):
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

def get_byol_transforms(size, mean, std):
    transformT = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomResizedCrop(size=size, scale=(0.08,1), ratio=(3 / 4, 4 / 3)),
        transforms.RandomRotation((-90, 90)),
        transforms.ColorJitter(),
        transforms.GaussianBlur(kernel_size=(23,23), sigma=(0.1, 2.0)),
        transforms.RandomGrayscale(p=0.2),
        transforms.Normalize(mean, std),
        ])

    transformT1 = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomResizedCrop(size=size, scale=(0.08,1), ratio=(3 / 4, 4 / 3)),
        transforms.RandomRotation((-90, 90)),
        transforms.ColorJitter(),
        transforms.RandomGrayscale(p=0.2),
        transforms.GaussianBlur(kernel_size=(23,23), sigma=(0.1, 2.0)),
        transforms.Normalize(mean, std),
        ])

    transformEvalT = transforms.Compose([
        transforms.ToTensor(),
        transforms.CenterCrop(size=size),
        transforms.Normalize(mean, std),        
    ])

    return transformT, transformT1, transformEvalT

def get_cxr_transforms(size=224, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):
    transform = transforms.Compose([
        transforms.RandomResizedCrop(size, scale=(0.3, 0.9), ratio=(3/4, 4/3)),
        transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=25, sigma=(0.1, 2.0))], p=0.5),
        transforms.Resize((256, 256)),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    transform_eval = transforms.Compose([
        transforms.ToTensor(),
        transforms.CenterCrop(size=size),
        transforms.Normalize(mean, std),        
    ])

    return transform, transform, transform_eval


class MultiViewDataInjector(object):
    def __init__(self, *args):
        self.transforms = args[0]
        self.random_flip = transforms.RandomHorizontalFlip()

    def __call__(self, sample, *with_consistent_flipping):
        if with_consistent_flipping:
            sample = self.random_flip(sample)
        output = [transform(sample) for transform in self.transforms]
        return output

In [3]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self,
    input_size = 2048,
    output_size = 8192,
    depth = 3,
    ):
        super().__init__()
        layers = []
        inp = input_size
        for d in range(depth):
            if d == depth - 1:
                layers.append(nn.Linear(inp, output_size))
            else:
                layers.extend([nn.Linear(inp, output_size), nn.BatchNorm1d(output_size), nn.ReLU(inplace=True)])
                inp = output_size
        self.layer = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layer(x)

class VicReg(nn.Module):
    def __init__(self,
    backend = 'resnet50',
    input_size = 2048,
    output_size = 8192,
    depth_projector = 3,
    lmbd = 5e-3, u = 1, v= 1, epsilon = 1e-3):

        super().__init__()
        self.backend = backend
        self.projector = MLP(input_size=input_size, output_size=output_size, depth=depth_projector)
        self.output_size = output_size
        self.epsilon = epsilon
        self.u = u
        self.v = v
        self.lmbd = lmbd
    
    def forward(self, x, y):
        x = self.backend(x)
        y = self.backend(y)
        x = self.projector(x)
        y = self.projector(y)

        bs = x.size(0)
    
        repr_loss = F.mse_loss(x, y)
    
        # x = torch.cat(FullGatherLayer.apply(x), dim=0)
        # y = torch.cat(FullGatherLayer.apply(y), dim=0)
        x = x - x.mean(dim=0)
        y = y - y.mean(dim=0)
    
        std_x = torch.sqrt(x.var(dim=0) + self.epsilon)
        std_y = torch.sqrt(y.var(dim=0) + self.epsilon)
        std_loss = torch.mean(F.relu(1 - std_x)) / 2 + torch.mean(F.relu(1 - std_y)) / 2
    
        cov_x = (x.T @ x) / (bs - 1)
        cov_y = (y.T @ y) / (bs - 1)
        cov_loss = off_diagonal(cov_x).pow_(2).sum().div(self.output_size) + off_diagonal(cov_y).pow_(2).sum().div(self.output_size)
    
        loss = (
            self.u * repr_loss
            + self.v * std_loss
            + self.lmbd * cov_loss
        )
        
        # Free tensors
        del x, y, cov_x, cov_y, std_x, std_y
        torch.cuda.empty_cache()

        return loss

In [4]:
# From https://github.com/noahgolmant/pytorch-lars
""" Layer-wise adaptive rate scaling for SGD in PyTorch! """
from torch.optim.optimizer import Optimizer, required

class LARS(Optimizer):
    r"""Implements layer-wise adaptive rate scaling for SGD.

    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): base learning rate (\gamma_0)
        momentum (float, optional): momentum factor (default: 0) ("m")
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
            ("\beta")
        eta (float, optional): LARS coefficient
        max_epoch: maximum training epoch to determine polynomial LR decay.

    Based on Algorithm 1 of the following paper by You, Gitman, and Ginsburg.
    Large Batch Training of Convolutional Networks:
        https://arxiv.org/abs/1708.03888

    Example:
        >>> optimizer = LARS(model.parameters(), lr=0.1, eta=1e-3)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """
    def __init__(self, params, lr=required, momentum=.9,
                 weight_decay=.0005, eta=0.001, max_epoch=200):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}"
                             .format(weight_decay))
        if eta < 0.0:
            raise ValueError("Invalid LARS coefficient value: {}".format(eta))

        self.epoch = 0
        defaults = dict(lr=lr, momentum=momentum,
                        weight_decay=weight_decay,
                        eta=eta, max_epoch=max_epoch)
        super(LARS, self).__init__(params, defaults)

    def step(self, epoch=None, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
            epoch: current epoch to calculate polynomial LR decay schedule.
                   if None, uses self.epoch and increments it.
        """
        loss = None
        if closure is not None:
            loss = closure()

        if epoch is None:
            epoch = self.epoch
            self.epoch += 1

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            eta = group['eta']
            lr = group['lr']
            max_epoch = group['max_epoch']

            for p in group['params']:
                if p.grad is None:
                    continue

                param_state = self.state[p]
                d_p = p.grad.data

                weight_norm = torch.norm(p.data)
                grad_norm = torch.norm(d_p)

                # Global LR computed on polynomial decay schedule
                decay = (1 - float(epoch) / max_epoch) ** 2
                global_lr = lr * decay

                # Compute local learning rate for this layer
                local_lr = eta * weight_norm / \
                    (grad_norm + weight_decay * weight_norm)

                # Update the momentum term
                actual_lr = local_lr * global_lr

                if 'momentum_buffer' not in param_state:
                    buf = param_state['momentum_buffer'] = \
                            torch.zeros_like(p.data)
                else:
                    buf = param_state['momentum_buffer']
                buf.mul_(momentum).add_(actual_lr, d_p + weight_decay * p.data)
                p.data.add_(-buf)

        return loss

## Pre-training

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset, random_split
import numpy as np
from tqdm import tqdm
import time

In [6]:
# Set params
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
epochs = 100
batch_size = 32
base_lr = 0.00025
weight_decay = 1e-6

In [7]:
# Load datasets
transform_x1, transform_x2, transform_test = get_cxr_transforms()

dataset_path = './datasets/chestx-ray14-v3'
train_dataset = datasets.ImageFolder(root=f'{dataset_path}/train', transform=MultiViewDataInjector([transform_x1, transform_x2]))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

test_dataset = datasets.ImageFolder(root=f'{dataset_path}/test', transform=MultiViewDataInjector([transform_x1, transform_x2]))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

In [12]:
# Train loop
def train_loop(model, epoch, optimizer, train_loader, device):
    tk0 = tqdm(train_loader)
    train_loss = []

    start_time = time.time()

    for (x, x1), _ in tk0:
        x = x.to(device)
        x1 = x1.to(device)

        loss = model.forward(x, x1)

        train_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step(epoch)

        # Free tensors
        del x, x1, loss
        torch.cuda.empty_cache()

    return train_loss, time.time() - start_time

In [9]:
from torchvision.models import efficientnet_b0

# Load model
model = efficientnet_b0(pretrained=True)
embed_dim = model.classifier[1].in_features
output_features = embed_dim*4
model.classifier[1] = nn.Identity()
model = VicReg(input_size=embed_dim, output_size=output_features, backend=model, depth_projector=3)
model

VicReg(
  (backend): EfficientNet(
    (features): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_activation

In [13]:
# Train script
start_time = time.time()

model = model.to(device)
params = model.parameters()
optimizer = LARS(params, lr=base_lr, weight_decay=weight_decay)

for epoch in range(epochs):
    train_loss, run_time = train_loop(model, epoch, optimizer, train_loader, device)
    print(f'Epoch: [{epoch+1}/{epochs}]  loss: {np.mean(train_loss)}, time: {run_time}')

end_time = time.time()
train_time = end_time - start_time
print(f'Training time {train_time}')

100%|██████████████████████████████████████████████████████████████████████████████| 378/378 [03:11<00:00,  1.97it/s]


Epoch: [1/100]  loss: 0.8958430943034944, time: 191.5934774875641


100%|██████████████████████████████████████████████████████████████████████████████| 378/378 [03:13<00:00,  1.95it/s]


Epoch: [2/100]  loss: 0.8951006262075334, time: 193.77315425872803


100%|██████████████████████████████████████████████████████████████████████████████| 378/378 [03:16<00:00,  1.93it/s]


...

Epoch: [98/100]  loss: 0.846441183140669, time: 199.840589761734


100%|██████████████████████████████████████████████████████████████████████████████| 378/378 [03:19<00:00,  1.89it/s]


Epoch: [99/100]  loss: 0.8461569287474193, time: 199.6111056804657


100%|██████████████████████████████████████████████████████████████████████████████| 378/378 [03:19<00:00,  1.89it/s]

Epoch: [100/100]  loss: 0.845874012778045, time: 199.61502313613892
Training time 19961.72344970703


In [14]:
# Save results
model_backbone_weights = model.backend
print(model_backbone_weights)

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [16]:
! mkdir ./output/VICReg

In [17]:
save_model_path = './output/VICReg'
torch.save({ 'model_state_dict': model_backbone_weights.state_dict() }, f'{save_model_path}/efficientnet_b0_backbone_weights.ckpt')

## Fine-tuning

In [18]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
import torch

# Path to the dataset
dataset_path = './datasets/COVIDGR_1.0'
positive_path = os.path.join(dataset_path, 'P')
negative_path = os.path.join(dataset_path, 'N')

# Data transformations
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.3, 0.9), ratio=(3/4, 4/3)),
    transforms.RandomApply(
            [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
            p=0.8
        ),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=25, sigma=(0.1, 2.0))], p=0.5),
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Load dataset with ImageFolder
dataset = datasets.ImageFolder(root=dataset_path, transform=train_transform)

In [19]:
# Step 4: Map labels
dataset.class_to_idx = {'N': 0, 'P': 1}
dataset

Dataset ImageFolder
    Number of datapoints: 852
    Root location: ./datasets/COVIDGR_1.0
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.3, 0.9), ratio=(0.75, 1.3333), interpolation=bilinear)
               RandomApply(
               p=0.8
               ColorJitter(brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.8, 1.2], hue=[-0.1, 0.1])
           )
               RandomGrayscale(p=0.2)
               RandomApply(
               p=0.5
               GaussianBlur(kernel_size=(25, 25), sigma=(0.1, 2.0))
           )
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )

In [20]:
# Splitting the dataset
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
batch_size = 256
print(train_size, batch_size)

# Split off the test set
train_val_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader for the test set (held out)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

766 256


In [21]:
checkpoint_path = "./output/VICReg/efficientnet_b0_backbone_weights.ckpt"
best_params = {"learning_rate": 0.01, "weight_decay": 0.001}

In [22]:
from torchvision.models import efficientnet_b0
import torch.nn as nn
import torch.optim as optim

def get_model():
    # Load the EfficientNet model
    model = efficientnet_b0()
    
    # Modify the final classification head for your dataset
    embed_dim = model.classifier[1].in_features
    model.classifier[1] = nn.Identity()
    
    # Load the pre-trained weights
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    state_dict = checkpoint["model_state_dict"]  # Adjust the key if needed
    
    # Load the weights into the model
    msg = model.load_state_dict(state_dict, strict=False)
    print('Pretrained weights found at {} and loaded with msg: {}'.format(checkpoint_path, msg))

    # Freeze model params
    for param in model.features.parameters():
        param.requires_grad = False
    return model, embed_dim

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time
from sklearn.model_selection import KFold

def calculate_metrics(true_labels, predictions):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    return accuracy, precision, recall, f1


def train_model(model, classifier, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    classifier = classifier.to(device)
    train_losses, val_losses = [], []
    best_val_f1 = 0.0

    for epoch in range(num_epochs):
        classifier.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            outputs = classifier(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()  # Adjust learning rate

        # Validation phase
        classifier.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                outputs = classifier(outputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Metrics
        accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
        train_losses.append(running_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))

        if f1 > best_val_f1:
            best_val_f1 = f1

        print(f"[{device}] Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss / len(train_loader):.4f}, "
              f"Val Loss: {val_loss / len(val_loader):.4f}, F1 Score: {f1:.4f}")

    return best_val_f1

# Evaluate model on the test set
def evaluate_model(model, classifier, test_loader):
    classifier.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    classifier = classifier.to(device)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            outputs = classifier(outputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')
    return accuracy, precision, recall, f1

def grid_search(train_loader, val_loader, learning_rates, weight_decays, num_epochs):
    best_model = None
    best_f1 = 0
    best_params = {}
    for lr in learning_rates:
        for wd in weight_decays:
            model = get_model()
            optimizer = optim.SGD(model.classifier[1].parameters(), lr=lr, weight_decay=wd, momentum=0.9)
            scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: min(1.0, (epoch + 1) / 10))
            criterion = nn.CrossEntropyLoss()
            print(f"\nTraining with lr={lr}, weight_decay={wd}")
            f1_score = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)
            if f1_score > best_f1:
                best_f1 = f1_score
                best_model = model
                best_params = {"learning_rate": lr, "weight_decay": wd}
    print(f"\nBest Model F1: {best_f1} with params {best_params}")
    return best_model, best_params

# Step 9: 5-Fold Cross-Validation
def cross_validation(best_params, dataset, test_loader, num_epochs=50, folds=5):
    fold_metrics = []
    kfold = KFold(n_splits=folds, shuffle=True, random_state=100)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"\nStarting fold {fold + 1}/{folds}")

        # Split dataset indices for training and validation
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Create DataLoaders for this fold
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        model, embed_dim = get_model()
        linear_classifier = nn.Linear(embed_dim, 2) # 2 is the number of features
        optimizer = optim.SGD(linear_classifier.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'], momentum=0.9)
        scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: min(1.0, (epoch + 1) / 10))
        criterion = nn.CrossEntropyLoss()

        start_time = time.time()
        _ = train_model(model, linear_classifier, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)
        end_time = time.time()

        # Evaluate on test set
        accuracy, precision, recall, f1 = evaluate_model(model, linear_classifier, test_loader)
        fold_metrics.append((accuracy, precision, recall, f1, end_time - start_time))

    return np.array(fold_metrics)

In [24]:
# Run 5-fold cross-validation
metrics = cross_validation(best_params, train_val_dataset, test_loader, num_epochs=50, folds=5)

# Calculate average and standard deviation of metrics across folds
avg_metrics = metrics.mean(axis=0)
std_metrics = metrics.std(axis=0)

print(f"\nAverage metrics over 5 folds in test set:\n"
      f"Accuracy: {avg_metrics[0]:.4f} ± {std_metrics[0]:.4f}\n"
      f"Precision: {avg_metrics[1]:.4f} ± {std_metrics[1]:.4f}\n"
      f"Recall: {avg_metrics[2]:.4f} ± {std_metrics[2]:.4f}\n"
      f"F1 Score: {avg_metrics[3]:.4f} ± {std_metrics[3]:.4f}\n"
      f"Training Time per Fold: {avg_metrics[4]:.2f} ± {std_metrics[4]:.2f} seconds")


Starting fold 1/5
Pretrained weights found at ./output/VICReg/efficientnet_b0_backbone_weights.ckpt and loaded with msg: <All keys matched successfully>
[cuda] Epoch 1/50, Train Loss: 0.6803, Val Loss: 0.7020, F1 Score: 0.4427
[cuda] Epoch 2/50, Train Loss: 0.6768, Val Loss: 0.6813, F1 Score: 0.4806
[cuda] Epoch 3/50, Train Loss: 0.6522, Val Loss: 0.6593, F1 Score: 0.6197
[cuda] Epoch 4/50, Train Loss: 0.6258, Val Loss: 0.6395, F1 Score: 0.6711
[cuda] Epoch 5/50, Train Loss: 0.6094, Val Loss: 0.6044, F1 Score: 0.7006
[cuda] Epoch 6/50, Train Loss: 0.5779, Val Loss: 0.5893, F1 Score: 0.7425
[cuda] Epoch 7/50, Train Loss: 0.5604, Val Loss: 0.6093, F1 Score: 0.6456
[cuda] Epoch 8/50, Train Loss: 0.5432, Val Loss: 0.5691, F1 Score: 0.7284
[cuda] Epoch 9/50, Train Loss: 0.5616, Val Loss: 0.6195, F1 Score: 0.7329
[cuda] Epoch 10/50, Train Loss: 0.5659, Val Loss: 0.6165, F1 Score: 0.7237
[cuda] Epoch 11/50, Train Loss: 0.5312, Val Loss: 0.6187, F1 Score: 0.6708
[cuda] Epoch 12/50, Train Loss

In [25]:
metrics

array([[7.55813953e-01, 6.97674419e-01, 7.89473684e-01, 7.40740741e-01,
        1.88389540e+03],
       [6.97674419e-01, 6.57894737e-01, 6.57894737e-01, 6.57894737e-01,
        1.86215698e+03],
       [6.86046512e-01, 6.27906977e-01, 7.10526316e-01, 6.66666667e-01,
        1.85964647e+03],
       [6.86046512e-01, 6.27906977e-01, 7.10526316e-01, 6.66666667e-01,
        1.83039524e+03],
       [6.86046512e-01, 6.48648649e-01, 6.31578947e-01, 6.40000000e-01,
        1.86942247e+03]])

In [26]:
import pandas as pd

results = pd.DataFrame(metrics, columns=['accuracy', 'precision', 'recall', 'f1', 'training time'])
results.head()

,accuracy,precision,recall,f1,training time
0,0.755814,0.697674,0.789474,0.740741,1883.895396
1,0.697674,0.657895,0.657895,0.657895,1862.156977
2,0.686047,0.627907,0.710526,0.666667,1859.646466
3,0.686047,0.627907,0.710526,0.666667,1830.395243
4,0.686047,0.648649,0.631579,0.640000,1869.422467


In [28]:
results.to_csv('./results/vicreg_fine_tuned.csv')